## Behavioral Cloning - Nvidia's "End to End Learning for Self-Driving Cars" Model

This notebook implement the mode based on the description [here](http://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf).


In [1]:
import os
import csv
import numpy as np
import cv2
import sklearn
import matplotlib.pyplot as plt

#height = 80
#width = 160
height = 160
width = 320

dataSubDir = 'minimum'
dataLog = 'driving_log.csv'
dataDir = os.path.join('data', dataSubDir)

samples = []
with open(os.path.join(dataDir,dataLog)) as csvfile:
    samples = list(csv.reader(csvfile))

train_samples = samples
validation_samples = samples
    
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        #shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = os.path.join(dataDir,batch_sample[0].strip())
                center_img = cv2.imread(name)
                center_image = cv2.resize(center_img,(width,height))
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            #yield sklearn.utils.shuffle(X_train, y_train)
            yield X_train, y_train

train_generator = generator(samples)
validation_generator = generator(samples)
print('generators setup')

generators setup


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, Lambda
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D

Using TensorFlow backend.


In [3]:
model = Sequential()
model.add(Lambda(lambda x: x/127.5 - 1,input_shape=(height, width, 3)))
model.add(Conv2D(24,5,5))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(36,5,5))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(48,5,5))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,3,3))
model.add(Conv2D(64,3,3))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))
model.compile('adam', loss='mse' , metrics=['accuracy'])
#model.summary()

### Run the following line when you want train the network


In [4]:
history = model.fit_generator(train_generator, \
                              samples_per_epoch = len(train_samples), \
                              validation_data=validation_generator, \
                              nb_val_samples=len(validation_samples), nb_epoch=10, verbose=0)

loss = history.history['loss']
acc = history.history['acc']
val_loss = history.history['val_loss']
val_acc = history.history['val_acc']

for i in range(len(loss)):
    print('{0:.4f} {1:.4f} {2:.4f} {3:.4f}'.format(loss[i], acc[i], val_loss[i], val_acc[i]))
    
predict = model.predict_generator(train_generator, val_samples=len(train_samples))

gen = generator(samples)
x, y = next(gen)
for i in range(len(train_samples)):
    print('{0:+.4f} {1:+.4f}'.format(y[i], predict[i][0]))


0.6316 0.3333 0.3376 0.3333
0.3376 0.3333 10.5807 0.0000
10.5807 0.0000 0.1563 0.6667
0.1563 0.6667 0.3969 0.3333
0.3969 0.3333 0.4137 0.3333
0.4137 0.3333 0.4214 0.3333
0.4214 0.3333 0.3043 0.3333
0.3043 0.3333 0.5249 0.3333
0.5249 0.3333 3.3683 0.0000
3.3683 0.0000 1.1774 0.0000
+0.0000 -1.0192
+1.0000 +2.4495
-0.9427 -1.5691
